This notebook trains models to calculate propensity scores.

Meaning, train a model to tell which of two datasets a sample came from.

If the sets are indistinguishable, a well-trained model should not perform better than a naive guess (half, if made to be balanced).


## Settings


In [1]:
# Whether to include the answers to questions when comparing elements from the datasets.
EXCLUDE_QUESTION_ANSWERS: bool = False

## Utilities


In [2]:
# Standard to handle notebooks being stored in a subdirectory
import os
import sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))

In [3]:
from truthfulqa_dataset import load_truthfulqa
import datasets
import numpy as np

## Load data


In [4]:
def get_truthfulqa_dataset_texts(
    truthfulqa_dataset: datasets.Dataset,
    exclude_choices: bool = EXCLUDE_QUESTION_ANSWERS,
) -> np.array:
    """
    Get the texts from a dataset that uses the TruthfulQA structure.

    Args:
        truthfulqa_dataset (datasets.Dataset):
            The dataset to get the texts from.
        exclude_choices (bool, optional): If this is True, only the
            questions will be embedded. If this is False, the questions
            and choices will be embedded. Defaults to False.
    """
    if exclude_choices:
        return truthfulqa_dataset["question"]
    else:
        return [
            "\n".join([x["question"]] + sorted(x["mc1_targets"]["choices"]))
            for x in truthfulqa_dataset
        ]

In [5]:
# 1. Load datasets
# @TODO Make utilities for these.

truthful_dataset = load_truthfulqa("misconceptions")
crafted_ds = datasets.load_dataset(
    "json", data_files="../datasets/crafted_dataset_unfiltered.jsonl"
)["train"]
generated_ds = datasets.load_dataset(
    "csv", data_files="../datasets/generated_dataset_unfiltered.csv"
)["train"]


def array(x, dtype=None):
    return x


# Special logic due to how the CSV stores choices as a string
generated_ds = generated_ds.map(
    lambda x: {
        "question": x["question"],
        "mc1_targets": eval(x["mc1_targets"], dict(globals(), array=array), locals()),
    }
)

dss = [truthful_dataset, crafted_ds, generated_ds]
dss_names = ["Orig", "Craft", "Gen"]

print("Dataset shapes", [ds.shape for ds in dss])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Dataset shapes [(100, 3), (24, 2), (99, 3)]


In [6]:
# To deal with imbalance, we'll just repeat the datasets a few times.
crafted_ds = datasets.concatenate_datasets([crafted_ds] * 5).select(
    range(truthful_dataset.shape[0])
)
generated_ds = datasets.concatenate_datasets([generated_ds] * 2).select(
    range(truthful_dataset.shape[0])
)

truthful_dataset = truthful_dataset.remove_columns(["mc1_targets", "mc2_targets"])
crafted_ds = crafted_ds.remove_columns(["mc1_targets"])
generated_ds = generated_ds.remove_columns(["mc1_targets"])

## Dataset prep


In [13]:
ds1 = truthful_dataset
ds2 = crafted_ds
# ds2 = generated_ds

# ds1 = truthful_dataset.select(range(50))
# ds2 = truthful_dataset.select(range(50, 100))

ds1 = ds1.add_column("label", [0] * ds1.shape[0])
ds2 = ds2.add_column("label", [1] * ds2.shape[0])


# combined_ds = datasets.concatenate_datasets([truthful_dataset, crafted_ds])
combined_ds = datasets.concatenate_datasets([ds1, ds2])

texts = get_truthfulqa_dataset_texts(combined_ds, exclude_choices=True)
combined_ds = combined_ds.add_column("text", texts)

In [29]:
# traintest_ds = combined_ds.train_test_split(test_size=0.2)

# Create different folds for cross-validation.
# This is so that every sample is present in the test set for some fold,
# and so the whole set used for analysis.

num_folds = 5

combined_ds = combined_ds.shuffle(seed=0)

cross_validation_datasets = [
    datasets.DatasetDict(
        {
            "train": combined_ds.select(
                [i for i in range(combined_ds.shape[0]) if i % 5 != j]
            ),
            "test": combined_ds.select(
                [i for i in range(combined_ds.shape[0]) if i % 5 == j]
            ),
        }
    )
    for j in range(num_folds)
]

In [30]:
# Basic transformers classification
# https://huggingface.co/docs/transformers/en/tasks/sequence_classification


import transformers
from transformers import AutoTokenizer

import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

import torch
from scipy.special import softmax


def finetune_propensity(
    traintest_ds: datasets.DatasetDict,
    model_name: str = "distilbert-base-cased",
    epochs: int = 50,
    save_name: str | None = None,
) -> transformers.Trainer:
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def preprocess_function(examples):
        return tokenizer(examples["text"], truncation=True)

    tokenized_dataset = traintest_ds.map(preprocess_function, batched=True)

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    accuracy = evaluate.load("accuracy")
    mse = evaluate.load("mse")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        probabilities = softmax(logits, axis=1)
        predictions = np.argmax(probabilities, axis=1)
        confidence_scores = probabilities[np.arange(len(predictions)), predictions]
        propensity_scores = (confidence_scores - 0.5) ** 2
        accuracy_score = accuracy.compute(predictions=predictions, references=labels)[
            "accuracy"
        ]
        mse_score = mse.compute(predictions=predictions, references=labels)["mse"]

        return {
            "accuracy": accuracy_score,
            "mse": mse_score,
            "mean_propensity_score": np.mean(propensity_scores),
        }

    from transformers import (
        AutoModelForSequenceClassification,
        TrainingArguments,
        Trainer,
    )

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2,
        # id2label=id2label, label2id=label2id
    )

    training_args = TrainingArguments(
        output_dir="./results",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=epochs,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="no",
        load_best_model_at_end=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    if save_name is not None:
        trainer.save_model(save_name)

    return trainer

In [32]:
for i, traintest_ds in enumerate(cross_validation_datasets):
    finetune_propensity(
        traintest_ds, save_name=f"propensity_orig_crafted-{i}", epochs=50
    )

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/500 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/3 [00:00<?, ?it/s]

...

/home/cwenner/miniconda3/envs/retrospective-llm-eval/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'eval_loss': 0.0004867399111390114, 'eval_accuracy': 1.0, 'eval_mse': 0.0, 'eval_mean_propensity_score': 0.2495143711566925, 'eval_runtime': 0.0307, 'eval_samples_per_second': 1304.017, 'eval_steps_per_second': 97.801, 'epoch': 49.0}
{'loss': 0.0537, 'learning_rate': 0.0, 'epoch': 50.0}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.00048649535165168345, 'eval_accuracy': 1.0, 'eval_mse': 0.0, 'eval_mean_propensity_score': 0.2495146095752716, 'eval_runtime': 0.0257, 'eval_samples_per_second': 1554.67, 'eval_steps_per_second': 116.6, 'epoch': 50.0}
{'train_runtime': 12.8489, 'train_samples_per_second': 622.624, 'train_steps_per_second': 38.914, 'train_loss': 0.053682758331298826, 'epoch': 50.0}


/home/cwenner/miniconda3/envs/retrospective-llm-eval/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
